# Linearna regresija

**"Linear approach to modelling the relationship between a scalar response and one or more explanatory variables"**

# Pregled
* Nadgledano učenje
* Prosta linearna regresija
* Višestruka linearna regresija: više feature-a
* Polinomijalna regresija: polinom umesto linearne funkcije

# Nadgledano učenje

**Nadgledano učenje** (induktivno učenje, supervised learning) se bavi problemima u mašinskom učenju gde treba otkriti vezu između ulaznih i izlaznih podataka tj. "naučiti" funkciju koja mapira ulaze na izlaze, na osnovu datog skupa parova $(ulaz, izlaz)$. Tu funkciju nazivamo **hipoteza**. 

Dva glavna tipa problema su **regresija** i **klasifikacija**.

[Više o nadgledanom učenju](https://mcerovic.github.io/notes/SupervisedLearning/index.html)




# Linearna regresija

* **Linearna regresija** (Linear Regression) je jedna od najpoznatijih metoda nadgledanog učenja
* Za početak bavićemo se **prostom** linearnom regresijom  (simple LR, univariate LR)
  * Imamo jednu ulaznu promenljivu (**feature**) i jednu izlaznu promenljivu
* **Notacija**:
  * $x$: Ulazni podaci (features), $x^{(i)}$: Ulazni podatak iz i-tog trening primera
  * $y$: Izlazni podaci (labels), $y^{(i)}$: Izlazni podatak iz i-tog trening primera
  * $m$: Veličina trening skupa - broj semplova
* **Hipoteza (model)**: $ h_\theta(x) = \theta_0 + \theta_1 x $
  * Linearna funkcija!
  * "Učimo" parametre $\theta_0$ i $\theta_1$, tj. modifikujemo ih tako da budu "što bolji"
  * Kako znamo kada su "bolji"?
* **Funkcija troška** (kriterijumska funkcija, funkcija koštanja, cost function, loss function)
  * U ovom slučaju za funkciju troška uzimamo **MSE** (srednje kvadratno odstupanje, mean squared error)
  * $J(\theta_0, \theta_1) = \frac{1}{m} \sum\limits_{i=1}^{m} (h_\theta(x^{(i)} - y^{(i)})^2$ (nekada $\frac{1}{2m}$ zbog lepšeg izvoda)
 * Dakle, menjamo $\theta_0$ i $\theta_1$ tako da **minimizujemo** funkciju troška, kako?
* **Optimizacija: gradijentni spust** (gradient descent): iterativni optimizacioni metod za minimizaciju funkcije
    * Određuje u kom smeru treba da se krećemo da što efikasnije poboljšamo vrednosti parametara, varijante: 
        * **Stochastic**: Računamo funkciju troška za svaki trening primer i ažuriramo parametre jednom po primeru
        * **Batch**: Agregiramo funkciju troška za sve primere u trening skupu i ažuriramo parametre jednom za ceo skup
        * **Minibatch**: Delimo trening skup na manje skupove (*batches*) fiksne veličine, i nad njima agregiramo funkciju troška, jedno ažuriranje parametara po batch-u
* [Više o linearnoj regresiji](https://mcerovic.github.io/notes/LinearRegression/index.html)
* [Više o gradijentnom spustu](https://mcerovic.github.io/notes/GradientDescent/index.html)
* [Više o 3 varijante gradijentnog spusta](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size/)

# Primer: predviđanje očekivanog životnog veka (OŽV) na osnovu nataliteta

Istraživanje od strane World Bank:  [World Development Indicators](https://datacatalog.worldbank.org/dataset/world-development-indicators).

[Prateća vizualizacija](https://www.google.com/publicdata/explore?ds=d5bncppjof8f9_&ctype=b&strail=false&nselm=s&met_x=sp_dyn_le00_in&scale_x=lin&ind_x=false&met_y=sp_dyn_tfrt_in&scale_y=lin&ind_y=false&met_s=sp_pop_totl&scale_s=lin&ind_s=false&dimp_c=country:region&ifdim=country&iconSize=0.5&uniSize=0.035#!ctype=b&strail=false&bcs=d&nselm=s&met_x=sp_dyn_le00_in&scale_x=lin&ind_x=false&met_y=sp_dyn_tfrt_in&scale_y=lin&ind_y=false&met_s=sp_pop_totl&scale_s=lin&ind_s=false&dimp_c=country:region&ifdim=country&pit=1421395200000&hl=en_US&dl=en_US&ind=false)

Ovo ukazuje na određenu korelaciju (veći natalitet => manji životni vek). Pokušaćemo da kvantifikujemo ovaj odnos tj. da naučimo funkciju $OŽV = f(natalitet)$.


In [0]:
%matplotlib inline
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np

tf.reset_default_graph()

# Korak 1: Učitavanje i obrada podataka.
# Često ćemo koristiti numpy zbog gotovih funkcija za parsiranje.
filename = 'birth_life_2010.tsv'
data = dict()
data['x'], data['y'] = np.loadtxt(filename, delimiter='\t', unpack=True, 
                                  skiprows=1, usecols=(1, 2))
# print(len(data['x']), len(data['y']))

# Nasumično mešanje podataka: uvek dobra ideja da se eliminiše bilo koja 
# pravilnost koja potencijalno postoji u ulaznom fajlu.
nb_samples = len(data['x'])
indices = np.random.permutation(nb_samples)
data['x'] = data['x'][indices]
data['y'] = data['y'][indices]

# Normalizacija: ubrzava gradijentni spust i čini sve promenljive ravnopravnim.
# Često dobra ideja, sada ćemo je izostaviti zarad jasnije vizualizacije.
# data['x'] = (data['x'] - np.mean(data['x'])) / np.std(data['x'])
# data['y'] = (data['y'] - np.mean(data['y'])) / np.std(data['y'])
plt.scatter(data['x'], data['y'])
plt.xlabel('Natalitet (prosečan broj dece po ženi)')
plt.ylabel('Očekivani životni vek')

# Korak 2: Model - linearna regresija.
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
theta_0 = tf.Variable(0.0)
theta_1 = tf.Variable(0.0)
hyp = tf.add(theta_0, tf.multiply(theta_1, X))

# Korak 3: Funkcija troška (MSE) i optimizacija.
# tf.train.Optimizer je porodica klasa koja generiše op-ove koji nemaju povratnu
# vrednost ali pri svakom pokretanju vrše iteraciju algoritma za minimizaciju
# određene promenljive.
loss = tf.reduce_mean(tf.square(hyp - Y))
opt_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

# Korak 4: Trening.
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  # Izvršavamo 100 epoha treninga (epoha = prolaz kroz sve podatke).
  nb_epochs = 100
  for epoch in range(nb_epochs):
    
    # Stochastic Gradient Descent (SGD).
    epoch_loss = 0
    for sample in range(nb_samples):
      feed = {X: data['x'][sample], Y: data['y'][sample]}
      _, curr_loss = sess.run([opt_op, loss], feed_dict=feed)
      epoch_loss += curr_loss
    
    # U svakoj desetoj epohi ispisujemo prosečan loss.
    epoch_loss /= nb_samples
    if (epoch + 1) % 10 == 0:
      print('Epoch: {}/{}| Avg loss: {:.2f}'.format(epoch+1, nb_epochs, 
                                              epoch_loss))
    
    # Dodajemo na grafik sve usputne hipoteze da prikažemo napredak.
    theta_0_val, theta_1_val = sess.run([theta_0, theta_1])
    xs = np.linspace(0, 8, 100)
    plt.plot(xs.tolist(), (theta_0_val + theta_1_val * xs).tolist(), 
            color=(1 - epoch / nb_epochs, epoch / nb_epochs, 0))
  
  # Ispisujemo i plotujemo finalnu vrednost parametara.
  theta_0_val, theta_1_val = sess.run([theta_0, theta_1])
  print('theta_0 = {:.2f}, theta_1 = {:.2f}'.format(theta_0_val, theta_1_val))
  xs = np.linspace(0, 8, 100)
  plt.plot(xs.tolist(), (theta_0_val + theta_1_val * xs).tolist(), color='g')
  plt.show()
  
  # Ispisujemo finalni MSE.
  final_loss = sess.run(loss, feed_dict={X: data['x'], Y: data['y']})
  print('Finalni loss: {:.5f}'.format(final_loss))

# Višestruka linearna regresija

* **Višestruka linearna regresija** (Multiple Linear Regression, nekad i Multivariate Linear Regression) je ime za linearnu regresiju gde radimo sa više od jednim ulaznim feature-om
* Kao primer radićemo sa datasetom sa sajta **kaggle** pod imenom [**80 cereals**](https://www.kaggle.com/crawford/80-cereals)
* Dati su razni podaci o 80 tipova cerealija sa kojima možemo raditi svašta... 
    * Recimo da želimo da predvidimo promenljivu **rating** na osnovu promenljivih **fat** i **sugars**


---



* Dodatna notacija: $x_i$ je i-ti feature, dakle $x_i^{(j)}$ je i-ti feature u j-tom trening primeru
* Hipoteza je sada: $ h_\theta(x) = w_0 + w_1 x_1 + ... + w_{n-1} x_{n-1} $
  * I dalje linearna funkcija samo po više promenlijvih; umesto fitovanja prave fitujemo ravan tj. hiperravan u određenom broju dimenzija
  * "Učimo" parametre $w$, $w_0$ (slobodan član) se često naziva *bias*
  * **Vektorizovana hipoteza**: $h_\theta(X)=X\Theta + bias$, gde je $X$ matrica $m\times n$ u kojoj je svaki red jedan trening primer, a $\Theta$ column-vektor dužine $n$
* Funkcija troška ostaje ista, kao i metod optimizacije


---

* Poći ćemo od prethodnog koda, i koristićemo vektorizovanu formulaciju tako da kod može da radi sa bilo kojim brojem trening primera
  * Moramo biti veoma pažljivi sa shape-ovima i broadcastingom
  * Pogledajmo primer pre glavnog koda

## Primer za broadcasting

In [0]:
# TODO

## 80 cereals višestruka linearna regresija

In [0]:
# TODO

# Polinomijalna regresija

* **Polinomijalna regresija** (Polynomial Regression) je naziv za regresiju u kojoj umesto linearne funkcije, učimo polinom nekog stepena
* Izraženo preko višestruke linearne regresije: od stepenova feature-a pravimo nove feature i onda radimo istu proceduru kao u višestrukoj regresiji
* Cela postavka dakle ostaje ista kao u višestrukoj linearnoj regresiji, samo je priprema feature-a drukčija
  * $ x_1 = x^1, x_2 = x^2, x_3 = x^3 \dots $
---
* Primenimo kod za višestruku regresiju na problem predviđanja **rating** pomoću **sugars** koristeći polinomijalnu regresiju i posmatrajmo kako stepen polinoma utiče na kvalitet rešenja
  * Polinomijalna regresija se mogla primeniti i na prvi primer (OŽV) ali su tu podaci po prirodi linearne prirode pa polinomi većeg stepena ne bi doneli značajna poboljšanja

In [0]:
# TODO